In [1]:
import bs4
import os
import re
import urllib.request

In [2]:
def split_camel_case_words(input_text):
    """
    Inside the given text splits camel case words into separate words.

    Parameters:
    - input_text (str): The text for cleaning the Camel case words.

    Requires:
    - This function requires the 're' module to be imported.

    Returns:
    - str: The input text with spaces added between camel case words.
    
    Example:
    >>> split_camel_case_words("CamelCase")
    'Camel Case'
    """
    output_text = re.sub(r"([a-z])([A-Z])", r"\1 \2", input_text)
    return output_text

In [3]:
def remove_repeating_empty_lines(input_text):
    """
    Removes repeating empty lines from the input text.

    Parameters:
    - input_text (str): The input text containing lines to be processed.

    Returns:
    - str: The input text without repeating empty lines.

    The function iterates through each line in the input text, 
    removing repeating empty lines. It keeps the first occurrence 
    of consecutive empty lines and removes subsequent ones. 
    The modified text is then returned.

    Example:
    >>> input_text = "Line 1\n\n\nLine 2\n\nLine 3\n\n\nLine 4"
    >>> remove_repeating_empty_lines(input_text)
    'Line 1\n\nLine 2\n\nLine 3\n\nLine 4'
    """
    lines = input_text.splitlines()
    modified_content = []

    for i in range(len(lines)):
        if i == len(lines) - 1 and len(lines) > 5:
            break;
        if lines[i].strip() != "":
            modified_content.append(lines[i])
        if lines[i].strip() == "" and lines[i+1].strip() != "":
            modified_content.append(lines[i])
            
    return '\n'.join(modified_content)

In [4]:
def translate_text(input_text, target_language="en", source_language="hr"):
    """
    Translates text from a source language to a target language.

    Parameters:
    - input_text (str): The text to be translated.
    - target_language (str): The target language code (default is "en" for English).
    - source_language (str): The source language code (default is "hr" for Croatian).

    Requires:
    - This function requires the followng import: google.cloud import translate_v2 as translate.

    Returns:
    - str: The translated text.

    This function uses the Google Cloud Translate API to translate each line
    of the input text from the specified source language to the target language.
    The resulting translated text is then returned. Note: Lines are used to preserve 
    empty text lines. 

    Example:
    >>> input_text = "Ovo je testni tekst."
    >>> translate_text(input_text, target_language="en", source_language="hr")
    'This is a test text.'
    """
    input_text_lines = input_text.splitlines()
    output_text_lines = []
    
    translate_client = translate.Client()
    translation_result = translate_client.translate(input_text_lines, target_language=target_language, source_language=source_language)

    for i in range(len(translation_result)):
        output_text_lines.append(translation_result[i]["translatedText"])
    
    return "\n".join(output_text_lines)

In [7]:
# list of all pages from www.algebra.hr that is useful for student administration office question
urls = [
    # "https://www.algebra.hr/sveuciliste/",
    # "https://www.algebra.hr/mba/hr/",
    # "https://www.algebra.hr/sveuciliste/hr/our-campus/",
    # "https://www.algebra.hr/algebra-spark-space/",
    # "https://www.algebra.hr/algebra-spark-space/spark-event-space/",
    # "https://www.algebra.hr/algebra-spark-space/algebra-spark-coworking-space/",
    # "https://www.algebra.hr/algebra-spark-space/rent-space/",
    # "https://www.algebra.hr/algebra-spark-space/spark-parking-space/",
    # "https://www.algebra.hr/mba/",
    # "https://www.algebra.hr/sveuciliste/kampanje/znanstvena-istrazivanja/",
    # "https://www.algebra.hr/sveuciliste/upisi/uvjeti-upisa/",
    # "https://www.algebra.hr/sveuciliste/upisi/kako-se-prijaviti/",
    # "https://www.algebra.hr/sveuciliste/upisi/dokumenti-za-upis/",
    # "https://www.algebra.hr/sveuciliste/upisi/upis-za-prijelaznike/",
    # "https://www.algebra.hr/sveuciliste/upisi/cijena-studija/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/kako-platiti-skolarinu-i-administrativne-naknade-putem-flywire-placanja/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/stipendije/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/financijske-potpore/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/studentski-krediti/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/financijske-potpore-za-poslodavce/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/stipendije/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/stipendije/stipendije-za-izvrsnost/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/stipendije/stipendije-za-vise-studijske-godine/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/stipendije/stipendije-udruge-hrvatsko-americkih-profesionalaca/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/stipendije/stipendije-iz-gospodarskog-fonda/",
    # "https://www.algebra.hr/sveuciliste/upisi/stipendije-i-financiranje/stipendije/stipendija-luka-mak/",
    # "https://www.algebra.hr/sveuciliste/kampanje/centar-karijera/",
    # "https://www.algebra.hr/sveuciliste/centar-karijera/poslodavci/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/korisne-informacije/ishodi-ucenja-na-razini-studijskog-programa/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/korisne-informacije/softverski-paket-ukljucen-u-cijenu-studija/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/korisne-informacije/tko-se-moze-upisati/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/korisne-informacije/termini-odvijanja-nastave/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/korisne-informacije/industrijski-certifikati/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/korisne-informacije/strucni-naziv-koji-se-stjece/",
    # "https://www.algebra.hr/sveuciliste/strucni-kratki-studij/internetski-marketing/korisne-informacije/trajanje-studija/",   
    # "https://www.algebra.hr/visoko-uciliste/preddiplomski-studij/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/razvoj-racunalnih-igara/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/razvoj-racunalnih-igara/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/razvoj-racunalnih-igara/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/programsko-inzenjerstvo/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/programsko-inzenjerstvo/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/programsko-inzenjerstvo/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/sistemsko-inzenjerstvo/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/sistemsko-inzenjerstvo/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/sistemsko-inzenjerstvo/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/multimedijska-produkcija/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/multimedijska-produkcija/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/multimedijska-produkcija/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/digitalni-marketing/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/digitalni-marketing/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/digitalni-marketing/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/digitalni-marketing/online-studij-za-izvanredne-studente/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/dizajn/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/dizajn/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/dizajn/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/3d-oblikovanje-trzisnih-komunikacija/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/3d-oblikovanje-trzisnih-komunikacija/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/prijediplomski-studij/3d-oblikovanje-trzisnih-komunikacija/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/programsko-inzenjerstvo/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/programsko-inzenjerstvo/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/programsko-inzenjerstvo/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/sistemsko-inzenjerstvo/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/sistemsko-inzenjerstvo/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/sistemsko-inzenjerstvo/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/razvoj-racunalnih-igara/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/razvoj-racunalnih-igara/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/razvoj-racunalnih-igara/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/podatkovni-smjer/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/podatkovni-smjer/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/podatkovni-smjer/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/podatkovni-smjer/data-science-and-mitx/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/digitalni-marketing/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/digitalni-marketing/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/digitalni-marketing/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/kreativno-upravljanje-trzisnim-komunikacijama/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/kreativno-upravljanje-trzisnim-komunikacijama/nastavni-plan/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/kreativno-upravljanje-trzisnim-komunikacijama/korisne-informacije/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/zdruzeni-diplomski-studij-racunarstva-internet-stvari-i-umjetna-inteligencija/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/zdruzeni-diplomski-studij-racunarstva-internet-stvari-i-umjetna-inteligencija/nastavni-program/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/zdruzeni-diplomski-studij-racunarstva-internet-stvari-i-umjetna-inteligencija/upisi/",
    # "https://www.algebra.hr/sveuciliste/diplomski-studij/",
    # "https://www.algebra.hr/sveuciliste/studenti-alumni-i-karijere/zdravstvena-zastita/",
]

In [ ]:
clean_txt_folder_path = "./data-web"

for i in range(len(urls)): 
    try:
        # download the raw web page html
        # extract texts from markup document
        web_page = str(urllib.request.urlopen(urls[i]).read().decode('utf-8', 'ignore'))
        soup = bs4.BeautifulSoup(web_page)

        # prepare file name to store original hr
        file_name = urls[i].replace("https://","").replace("/","-")
        file_name_hr = f"./{clean_txt_folder_path}/{file_name}hr.txt"

        # clean text by removing merged words
        # clean text by removing the multiple repeating empty lines
        text_hr = split_camel_case_words(soup.get_text())
        text_hr = remove_repeating_empty_lines(text_hr)
    
        # save text in hr
        with open(file_name_hr, "w") as file:
            file.write(text_hr)
            
    except Exception as e:
        print(f"Error with {urls[i]} - {e}")